In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("AutoLoaderExample").getOrCreate()

In [0]:
input_path = "/FileStore/tables/"

In [0]:
checkpoint_path = "/FileStore/checkpoints/autoloader_checkpoint"

In [0]:
output_table = "autoloader_output_table"

In [0]:
df = (spark.readStream
      .format("cloudFiles")  # Use Auto Loader's cloudFiles source
      .option("cloudFiles.format", "csv")  # Specify the file format
      .option("header", "true")  # Indicate the CSV has a header
      .schema("id INT, name STRING, age INT")  # Define the schema explicitly
      .load(input_path))

In [0]:
dbutils.fs.ls("dbfs:/user/hive/warehouse/autoloader_output_table")

Out[9]: [FileInfo(path='dbfs:/user/hive/warehouse/autoloader_output_table/_delta_log/', name='_delta_log/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/user/hive/warehouse/autoloader_output_table/part-00000-81a1f527-b9d0-4d8f-bc56-cd3b22425457-c000.snappy.parquet', name='part-00000-81a1f527-b9d0-4d8f-bc56-cd3b22425457-c000.snappy.parquet', size=1049, modificationTime=1742443886000),
 FileInfo(path='dbfs:/user/hive/warehouse/autoloader_output_table/part-00000-8c80c3a7-bd4a-443e-92fc-6aba310f7e5b-c000.snappy.parquet', name='part-00000-8c80c3a7-bd4a-443e-92fc-6aba310f7e5b-c000.snappy.parquet', size=956, modificationTime=1742443196000),
 FileInfo(path='dbfs:/user/hive/warehouse/autoloader_output_table/part-00000-aabd7bee-f75d-4b81-be9f-44a35b9f6093-c000.snappy.parquet', name='part-00000-aabd7bee-f75d-4b81-be9f-44a35b9f6093-c000.snappy.parquet', size=112165, modificationTime=1742443208000),
 FileInfo(path='dbfs:/user/hive/warehouse/autoloader_output_table/part-00001-07c88e53-43b4-4b0

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/autoloader_output_table", recurse=True)

Out[10]: True

In [0]:
checkpoint_path = "dbfs:/checkpoint/autoloader_output_table"

In [0]:
query = (df.writeStream
         .format("delta")
         .option("checkpointLocation", checkpoint_path)
         .outputMode("append")
         .table("autoloader_output_table"))